In [ ]:
%cd ..

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import tensorflow as tf

In [ ]:
import tensorflow_datasets as tfds
import tensorflow_datasets_bw as datasets
import eval_utils
from eval_utils import utils, nb_deblurring
from eval_utils.vsr import _load_vsr_dataset, _define_vsr_model, VSR_DATASETS

In [ ]:
import dppp

bicubic_rescale = dppp.create_resize_fn()

def superresolver(vid, idx, scale, _):
    return bicubic_rescale(vid[idx][None, ...], scale, True)[0]

In [ ]:
def example_vsr(superresolver, result, example_idx=0, save_gt=False, seed=99):
    example = EXAMPLES[example_idx]
    scale_factor = example["scale_factor"]
    image_idx = example["image_idx"]

    _, __, dataset = _load_vsr_dataset(example["dataset_def"], scale_factor)
    data = datasets.get_one_example(dataset, index=image_idx)

    model = _define_vsr_model(superresolver, scale_factor)
    hr, sr = model(data, seed + example_idx)
    print(hr.shape)

    # Save the example
    utils.save_example(
        gt=hr,
        output=sr,
        result=result,
        save_gt=save_gt,
        figure_width=1.1,  # TODO set according to size in pdf
        **example["figure_args"],
    )

In [ ]:
EXAMPLES = [
    {
        "dataset_def": {
            "name": "MDSP Color",
            "key": "mdsp_color_sr",
            "split": "test",
            "lr_image_key": "video",
        },
        "image_idx": 0,
        "scale_factor": 4,
        "figure_args": {
            "compute_metrics": False,
            "region_center": (50, 50),
            "region_extent": 50,
            "rel_width": 0,
            "place_border": 0.01,
            "place_top": False,
            "place_right": False,
        },
    },
]

with open("tmp.png", "w") as f:
    example_vsr(superresolver, f, save_gt=False)
